<h1> Pedram MohajerAnsari </h1>
<h2> Deep Learning        </h2>
<h2> Dr. Feng Luo         </h2>

In [ ]:
from google.colab import drive
drive.mount('content')

In [ ]:
cd '/content/content/MyDrive/Deep_Learning_HW3/'

In [ ]:
! ls

<h3> Install transformers </h3>

In [ ]:
!pip install packaging==21.3

In [ ]:
!pip install transformers==4.5.0

<h3> Import Package </h3>


In [ ]:
from transformers import AdamW, BertTokenizerFast, BertForQuestionAnswering

<h1> -> Task 1: Warm-up</h2>


<h3> Load Model and Tokenizer </h3>


In [ ]:
# model_name can be one of models in huggingface model hub 

model_name = 'bert-base-chinese'
model = BertForQuestionAnswering.from_pretrained(model_name)
eng_tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

<h3> Tokenize </h3>

In [ ]:
eng_paragraph = 'professor feng Deep Learning class'

tokens = eng_tokenizer.tokenize(eng_paragraph)
print(tokens)
eng_tokenizer.convert_tokens_to_ids(tokens)

<h3> Encode vs Decode </h3>

In [ ]:
question = 'Who established Apple?'
paragraph = 'Apple Computers, Inc. was founded on April 1, 1976, by college dropouts Steve Jobs and Steve Wozniak'

encoded = eng_tokenizer.encode(question, paragraph)
decoded = eng_tokenizer.decode(encoded)
print(encoded)
print(decoded)

<h3> Model Inputs </h3>

In [ ]:
inputs = eng_tokenizer(question, paragraph, return_tensors='pt') 

print('##################')
# Indices of input sequence tokens in the vocabulary
print('Input ids:      ', inputs['input_ids'])
print('##################')
# Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
print('Token type ids: ', inputs['token_type_ids'])
print('##################')
# Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
print('Attention mask: ', inputs['attention_mask'])

<h3> Training </h3>

<h4> > For Question Answering, loss is the sum of cross entropy between the model prediction and correct answer </h4>

In [ ]:
output = model(**inputs, start_positions=torch.tensor([14]), end_positions=torch.tensor([19]))
print("loss: ", output.loss)

optimizer = AdamW(model.parameters(), lr=1e-4)
output.loss.backward()
optimizer.step()

<h3> Testing </h3>

In [ ]:
question = 'when  Apple was founded?'
paragraph = 'Apple Computers, Inc. was founded on April 1, 1976, by college dropouts Steve Jobs and Steve Wozniak'

inputs = eng_tokenizer(question, paragraph, return_tensors='pt')

with torch.no_grad():
    output = model(**inputs)

print("start_logits: ")
print(output.start_logits)

print("end_logits: ")
print(output.end_logits)

start = torch.argmax(output.start_logits)
end = torch.argmax(output.end_logits)
print("start position: ", start.item())
print("end position:   ", end.item())

predict_id = inputs['input_ids'][0][start : end + 1]
print("predict_id:     ", predict_id)

predict_answer = eng_tokenizer.decode(predict_id)
print("predict_answer: ", predict_answer)

In [ ]:
####################################################################################
####################################################################################
####################################################################################
####################################################################################

<h1> -> Task 2:</h2>

## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn the process of using transformers to fine-tune a pretrained model.

- Tasks:
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing


## Import Packages

In [ ]:
import math
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
from transformers import get_linear_schedule_with_warmup

from tqdm.auto import tqdm

device = torch.device("cuda", 1) if torch.cuda.is_available() else "cpu"


def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

same_seeds(0)    # Fix random seed for reproducibility

In [ ]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)

fp16_training = True

if fp16_training:
    %pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 KB 5.8 MB/s eta 0:00:00


## Load Model and Tokenizer




 

In [ ]:
model = BertForQuestionAnswering.from_pretrained("bert-base-chinese").to(device)
tokenizer = BertTokenizerFast.from_pretrained("bert-base-chinese")


# model = AutoModelForQuestionAnswering.from_pretrained("luhua/chinese_pretrain_mrc_macbert_large").to(device)
# tokenizer = AutoTokenizer.from_pretrained("luhua/chinese_pretrain_mrc_macbert_large")



Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-chinese a

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/269k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

## Read Data

- Training set: 31690 QA pairs
- Dev set: 4131  QA pairs
- Test set: 4957  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [ ]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("train.json")
dev_questions, dev_paragraphs     = read_data("dev.json")
test_questions, test_paragraphs   = read_data("test.json")

print('train_questions : ', len(train_questions))
print('dev_questions   : ', len(dev_questions))
print('test_questions  : ', len(test_questions))

train_questions :  31690
dev_questions   :  4131
test_questions  :  4957


## Tokenize Data

In [ ]:
# Tokenize questions and paragraphs separately

train_questions_tokenized  = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized    = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized   = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized   = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized  = tokenizer(test_paragraphs, add_special_tokens=False)


## Dataset and Dataloader

In [ ]:
DOC_STRIDE = None

In [ ]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 40
        self.max_paragraph_len = 350
        
        ##### Change value of doc_stride #####
        self.doc_stride = int(0.9 * self.max_paragraph_len)

        ############################################
        global DOC_STRIDE
        
        DOC_STRIDE = self.doc_stride
        ############################################
        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]

        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### Preprocessing #####

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            mid = (answer_start_token + answer_end_token) // 2
            prefix_len = int(random.random() * self.max_paragraph_len)
            postfix_len = self.max_paragraph_len - prefix_len
            paragraph_start, paragraph_end = mid - prefix_len, mid + postfix_len
            if paragraph_start < 0:
                paragraph_end -= paragraph_start
                paragraph_start = 0
            if paragraph_end >= len(tokenized_paragraph):
                paragraph_end = len(tokenized_paragraph) - 1
            
            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation
        # Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 4

#########################

train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader   = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader  = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Function for Evaluation

In [ ]:
def evaluate(data, output, paragraph, paragraph_tokenized):
    ##### Postprocessing #####
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    
    paragraph_start_index = 0
    paragraph_end_index = 0
    
    for k in range(num_of_windows):

        mask = (data[1][0][k].bool() & data[2][0][k].bool()).to(device)
    
        masked_output_start = torch.masked_select(output.start_logits[k], mask)
        masked_output_start = masked_output_start[:-1]
        
        start_prob, start_index = torch.max(masked_output_start, dim=0)
        
        masked_output_end = torch.masked_select(output.end_logits[k], mask)
        masked_output_end = masked_output_end[start_index: -1]
        
        end_prob, end_index = torch.max(masked_output_end, dim=0)
        
        end_index += start_index
        
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        masked_data = torch.masked_select(data[0][0][k].to(device), mask)[:-1]
        
        # Replace answer if calculated probability is larger than previous windows
        if (prob > max_prob) and (start_index <= end_index <= (start_index + 50)):
            max_prob = prob
            paragraph_start_index = start_index.item() + (DOC_STRIDE * k)
            paragraph_end_index = end_index.item() + (DOC_STRIDE * k)
            answer = tokenizer.decode(masked_data[start_index : end_index + 1])


    
    ##########
    char_count = 0
    start_flag = False

    for i, token in enumerate(paragraph_tokenized):
        if token in ('[UNK]', '[CLS]', '[SEP]'):
            if i == paragraph_start_index:
                new_start = char_count
            if i == paragraph_end_index:
                new_end = char_count
            char_count += 1
        else:
            for char in token:
                if i == paragraph_start_index and not start_flag:
                    new_start = char_count
                    start_flag = True
                if i == paragraph_end_index:
                    new_end = char_count
                if char == "#":
                    continue
                else:
                    while char_count < len(paragraph) and char != paragraph[char_count]:
                        char_count += 1
                    char_count += 1
     
    if "[UNK]" in answer:
        print(f"original answer: {answer}")
        answer = paragraph[new_start: new_end+1]
        print(f"corrected answer: {answer}")
        print("-"*50)

###########################################################

    answer = answer.replace(' ', '')
    
###########################################################

    if len(answer) > 1:
        if "「" not in answer and answer[-1] == "」":
            answer = answer[:-1]
    return answer

## Training

In [ ]:
num_epoch     = 5  
validation    = True  
logging_step  = 500
learning_rate = 5e-6

optimizer = AdamW(model.parameters(), lr=learning_rate)

##### Apply linear learning rate decay #####
total_steps = len(train_loader) * num_epoch
# warmup_steps = int(0.05 * total_steps)

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

##################################################

if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 

model.train()

print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0
    
    for batch_idx, data in enumerate(tqdm(train_loader)):
        # Load all data into GPU
        data = [i.to(device) for i in data]
        
        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

        # Choose the most probable start position / end position
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)

        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
        train_loss += output.loss

        if fp16_training:
            accelerator.backward(output.loss)
        else:
            output.loss.backward()

        ##### Apply linear learning rate decay #####
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        step += 1
        ##################################################
        
        # Print training loss and accuracy over past logging step
        if step % logging_step == 0:
            print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            train_loss = train_acc = 0
            
    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output, dev_paragraphs[dev_questions[i]['paragraph_id']], dev_paragraphs_tokenized[dev_questions[i]['paragraph_id']].tokens) == dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()


##################################################

# Save a model 
print("Saving Model ...")
model_save_dir = "./models/saved_model" 
model.save_pretrained(model_save_dir)

Start Training ...


  0%|          | 0/7923 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch 1 | Step 500 | loss = 3.207, acc = 0.168
Epoch 1 | Step 1000 | loss = 1.595, acc = 0.456
Epoch 1 | Step 1500 | loss = 1.339, acc = 0.541
Epoch 1 | Step 2000 | loss = 1.239, acc = 0.553
Epoch 1 | Step 2500 | loss = 1.085, acc = 0.600
Epoch 1 | Step 3000 | loss = 1.027, acc = 0.621
Epoch 1 | Step 3500 | loss = 0.986, acc = 0.639
Epoch 1 | Step 4000 | loss = 0.959, acc = 0.630
Epoch 1 | Step 4500 | loss = 0.951, acc = 0.652
Epoch 1 | Step 5000 | loss = 0.891, acc = 0.657
Epoch 1 | Step 5500 | loss = 0.900, acc = 0.648
Epoch 1 | Step 6000 | loss = 0.841, acc = 0.686
Epoch 1 | Step 6500 | loss = 0.874, acc = 0.664
Epoch 1 | Step 7000 | loss = 0.869, acc = 0.673
Epoch 1 | Step 7500 | loss = 0.870, acc = 0.670
Evaluating Dev Set ...


  0%|          | 0/4131 [00:00<?, ?it/s]

...

original answer: 李 [UNK]
corrected answer: 李杲
--------------------------------------------------
original answer: 日 治 時 代 時 因 為 奧 運 進 場 時 「 [UNK] 」 比 「 [UNK] 」 先 進 場
corrected answer: 日治時代時因為奧運進場時「C」比「J」先進場
--------------------------------------------------
original answer: 朱 允 [UNK]
corrected answer: 朱允炆
--------------------------------------------------
original answer: [UNK] 崎 八 幡 宮
corrected answer: 筥崎八幡宮
--------------------------------------------------
original answer: 與 慕 容 [UNK] 雙 方 不 和
corrected answer: 與慕容廆雙方不和
--------------------------------------------------
original answer: 朱 允 [UNK] 有 「 毋 使 朕 有 殺 叔 父 名 」 旨 意
corrected answer: 朱允炆有「毋使朕有殺叔父名」旨意
--------------------------------------------------
original answer: [UNK] [UNK]
corrected answer: Du
--------------------------------------------------
original answer: 對 日 本 報 紙 的 無 恥 造 謠 誣 [UNK] ， 進 行 了 有 力 駁 斥
corrected answer: 對日本報紙的無恥造謠誣衊，進行了有力駁斥
--------------------------------------------------
original answer: 杜 恆 - [UNK] 因

  0%|          | 0/7923 [00:00<?, ?it/s]

Epoch 5 | Step 500 | loss = 0.337, acc = 0.840
Epoch 5 | Step 1000 | loss = 0.334, acc = 0.848
Epoch 5 | Step 1500 | loss = 0.330, acc = 0.854
Epoch 5 | Step 2000 | loss = 0.300, acc = 0.864
Epoch 5 | Step 2500 | loss = 0.306, acc = 0.848
Epoch 5 | Step 3000 | loss = 0.314, acc = 0.851
Epoch 5 | Step 3500 | loss = 0.363, acc = 0.833
Epoch 5 | Step 4000 | loss = 0.326, acc = 0.854
Epoch 5 | Step 4500 | loss = 0.354, acc = 0.828
Epoch 5 | Step 5000 | loss = 0.309, acc = 0.852
Epoch 5 | Step 5500 | loss = 0.338, acc = 0.842
Epoch 5 | Step 6000 | loss = 0.317, acc = 0.854
Epoch 5 | Step 6500 | loss = 0.353, acc = 0.843
Epoch 5 | Step 7000 | loss = 0.320, acc = 0.851
Epoch 5 | Step 7500 | loss = 0.320, acc = 0.849
Evaluating Dev Set ...


  0%|          | 0/4131 [00:00<?, ?it/s]

original answer: 李 [UNK]
corrected answer: 李杲
--------------------------------------------------
original answer: 日 治 時 代 時 因 為 奧 運 進 場 時 「 [UNK] 」 比 「 [UNK] 」 先 進 場
corrected answer: 日治時代時因為奧運進場時「C」比「J」先進場
--------------------------------------------------
original answer: 朱 允 [UNK]
corrected answer: 朱允炆
--------------------------------------------------
original answer: [UNK] 崎 八 幡 宮
corrected answer: 筥崎八幡宮
--------------------------------------------------
original answer: 與 慕 容 [UNK] 雙 方 不 和
corrected answer: 與慕容廆雙方不和
--------------------------------------------------
original answer: 朱 允 [UNK] 有 「 毋 使 朕 有 殺 叔 父 名 」 旨 意
corrected answer: 朱允炆有「毋使朕有殺叔父名」旨意
--------------------------------------------------
original answer: [UNK] [UNK]
corrected answer: Br
--------------------------------------------------
original answer: 對 日 本 報 紙 的 無 恥 造 謠 誣 [UNK] ， 進 行 了 有 力 駁 斥
corrected answer: 對日本報紙的無恥造謠誣衊，進行了有力駁斥
--------------------------------------------------
original answer: 杜 恆 - [UNK] 因

## View Results

In [ ]:
model.eval()
with torch.no_grad():
    dev_acc = 0
    for i, data in enumerate(tqdm(dev_loader)):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
               attention_mask=data[2].squeeze(dim=0).to(device))
        # prediction is correct only if answer text exactly matches
        pred_answer = evaluate(data, output, dev_paragraphs[dev_questions[i]['paragraph_id']], dev_paragraphs_tokenized[dev_questions[i]['paragraph_id']].tokens)
        true_answer = dev_questions[i]["answer_text"]
        dev_acc += (pred_answer == true_answer)
        if pred_answer != true_answer:
            print("*"*50)
            print(f"correct answer: {true_answer}")
            print(f"predict answer: {pred_answer}")
            print("*"*50)
    print(f"Validation | acc = {dev_acc / len(dev_loader):.3f}")
model.train()
print(f"Validation | acc = {dev_acc / len(dev_loader):.3f}")

  0%|          | 0/4131 [00:00<?, ?it/s]

**************************************************
correct answer: 納妾制度
predict answer: 納妾
**************************************************
**************************************************
correct answer: 三
predict answer: 第三長河
**************************************************
**************************************************
correct answer: 康有為故意不發軍餉造成
predict answer: 擅作主張分散兵力
**************************************************
**************************************************
correct answer: 南京
predict answer: 南京政府
**************************************************
**************************************************
correct answer: 「此心光明，亦復何言！」
predict answer: 病逝
**************************************************
**************************************************
correct answer: 「文化大革命」
predict answer: 文化大革命
**************************************************
**************************************************
correct answer: 浙江省
predict answer: 江蘇省
*******************************

## Testing

In [ ]:
print("Evaluating Test Set ...")

result = []

model.eval()
with torch.no_grad():
    for i, data in enumerate(tqdm(test_loader)):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output, test_paragraphs[test_questions[i]['paragraph_id']], test_paragraphs_tokenized[test_questions[i]['paragraph_id']].tokens))

result_file = "./result.csv"
with open(result_file, 'w') as f:
    f.write("ID,Answer\n")
    for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


  0%|          | 0/4957 [00:00<?, ?it/s]

original answer: 自 大 型 購 物 中 心 [UNK] 開 幕 後
corrected answer: 自大型購物中心MegaBox開幕後
--------------------------------------------------
original answer: 溥 [UNK]
corrected answer: 溥儁
--------------------------------------------------
original answer: 目 前 沒 有 觀 察 到 任 何 語 言 純 [UNK] 以 力 道 來 區 分 不 同 輔 音
corrected answer: 目前沒有觀察到任何語言純綷以力道來區分不同輔音
--------------------------------------------------
original answer: [UNK] 人 國 ； 公 元 前 28 年 ， 滅 北 沃 沮
corrected answer: 荇人國；公元前28年，滅北沃沮
--------------------------------------------------
original answer: [UNK]
corrected answer: 2
--------------------------------------------------
original answer: 馬 [UNK]
corrected answer: 馬馼
--------------------------------------------------
original answer: 東 晉 常 [UNK]
corrected answer: 東晉常璩
--------------------------------------------------
original answer: 校 長 [UNK]. [UNK] [UNK]. von [UNK]
corrected answer: SC校長Dr. Rufus B. von 
--------------------------------------------------
original answer: [UNK] 稻
corrected answer: